<a href="https://colab.research.google.com/github/zahraDehghanian97/DP-FedAvg/blob/master/DP_FedAvg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FedAvg

simulate a distributed learning scenario with multiple clients and a central server.

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler

# Load CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Partition the dataset across clients
num_clients = 4
client_indices = [list(range(i * len(trainset) // num_clients, (i + 1) * len(trainset) // num_clients)) for i in range(num_clients)]

# Create DataLoaders for each client
client_loaders = [DataLoader(trainset, batch_size=64, sampler=SubsetRandomSampler(indices)) for indices in client_indices]

# Create a DataLoader for the test dataset
test_loader = DataLoader(testset, batch_size=64, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


implement FedAvg

In [5]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_client(client_loader, global_model, local_epochs=1):
    local_model = SimpleCNN()
    local_model.load_state_dict(global_model.state_dict())
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(local_model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(local_epochs):
        for i, data in enumerate(client_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = local_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    return local_model.state_dict()

def aggregate_models(global_model, client_models, client_loaders):
    total_samples = sum([len(loader.dataset) for loader in client_loaders])
    avg_state_dict = global_model.state_dict()

    for key in avg_state_dict.keys():
        avg_state_dict[key] = torch.stack([client_models[i][key] * len(client_loaders[i].dataset) / total_samples for i in range(len(client_models))], dim=0).sum(dim=0)

    global_model.load_state_dict(avg_state_dict)
    return global_model

def test_model(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

train model on fixed number of rounds

In [ ]:
global_model = SimpleCNN()
num_rounds = 20
local_epochs = 1

for round in range(num_rounds):
    client_models = [train_client(client_loader, global_model, local_epochs) for client_loader in client_loaders]
    global_model = aggregate_models(global_model, client_models, client_loaders)
    accuracy = test_model(global_model, test_loader)
    print(f"Round {round + 1}: Global model accuracy: {accuracy:.4f}")

#DP-FedAvg

In [7]:
!pip install tensorflow_privacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.4/359.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.6/532.6 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.4 MB/s eta 0:00:00
  Attempting uninstall: mpmath
    Found existing installation: mpmath 1.3.0
    Uninstalling mpmath-1.3.0:
      Successfully uninstalled mpmath-1.3.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1

In [8]:
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer

def train_client_dp(client_loader, global_model, local_epochs=1, epsilon=1.0, delta=1e-5):
    local_model = SimpleCNN()
    local_model.load_state_dict(global_model.state_dict())
    criterion = nn.CrossEntropyLoss()
    
    # Use DP-SGD optimizer for differential privacy
    optimizer = DPGradientDescentGaussianOptimizer(
        l2_norm_clip=1.0,
        noise_multiplier=epsilon,
        num_microbatches=1,
        learning_rate=0.001,
        population_size=len(client_loader.dataset),
        delta=delta
    )

    for epoch in range(local_epochs):
        for i, data in enumerate(client_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = local_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    return local_model.state_dict()

tune parameter and train model on same number of rounds 

In [9]:
global_model_dp = SimpleCNN()
epsilon = 1.0
delta = 1e-5

for round in range(num_rounds):
    client_models = [train_client_dp(client_loader, global_model_dp, local_epochs, epsilon, delta) for client_loader in client_loaders]
    global_model_dp = aggregate_models(global_model_dp, client_models, client_loaders)
    accuracy = test_model(global_model_dp, test_loader)
    print(f"Round {round + 1}: Global model accuracy (DP-FedAvg): {accuracy:.4f}")

TypeError: ignored